# Word text generation: Leo Tolstoy "War and peace"

In [1]:
import sys
import random
import numpy as np

from keras.callbacks import *
from keras.optimizers import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, Bidirectional, LSTM, Embedding

with open('data/voina-i-mir.txt', encoding='utf-8') as f:
    text = f.read().lower().replace('\n', ' \n ')
    
start = text.index('часть первая')
finish = text.index('часть четвертая')

text = text[start:finish]   

Using TensorFlow backend.


## Getting words from text

In [2]:
import re
def text_rewiew(text):
    text = re.sub('[^а-яa-z]+', ' ', text)
    return text.split()

text_in_words= text_rewiew(text)

words = sorted(set(text_in_words))

## Get word sequences

In [3]:
sentences = []
next_words = []

SEQUENCE_LEN = 12

for i in range(0, len(text_in_words) - SEQUENCE_LEN, 1):
    sentences.append(text_in_words[i: i + SEQUENCE_LEN])
    next_words.append(text_in_words[i + SEQUENCE_LEN])

print('Num sequences:', len(sentences))
print('Input example:')
print(' '.join([sentences[0][i] for i in range(len(sentences[0]))]), '->', next_words[0])

Num sequences: 180530
Input example:
часть первая i eh bien mon prince g nes et lucques ne -> sont


In [4]:
word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

word_indices['bref']

251

Creating X and Y sets: we will predict the las word in sentece

In [5]:
x = np.zeros([len(sentences), SEQUENCE_LEN], dtype=np.int)
y = np.zeros([len(sentences)], dtype=np.int)

for i, sentence in enumerate(sentences):
    if len(sentence) > 2:
        for t, word in enumerate(sentence[:-1]):
            x[i, t] = word_indices[word]
        y[i] = word_indices[sentence[-1]]       

In [6]:
x.shape

(180530, 12)

In [7]:
y.shape

(180530,)

## Build and fit recurrent neuro model

In [8]:
from keras.layers.wrappers import Bidirectional

model = Sequential()
model.add(Embedding(input_dim=len(words), output_dim=1024, input_length=SEQUENCE_LEN))
model.add(Bidirectional(LSTM(units=128)))
model.add(Dense(units=len(words), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

### Перед тем как предсказывать, проверь, есть ли слово в словаре

In [8]:
'мной' in w2v_model.wv.vocab

True

Define callbacks and fit model

In [9]:
def sample(preds, temperature=1.0):
    if temperature <= 0:
        return np.argmax(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_next(text, num_generated=10):
    word_idxs = [word2idx(word) for word in text.lower().split()]
    for i in range(num_generated):
        prediction = model.predict(x=np.array(word_idxs))
        idx = sample(prediction[-1], temperature=0.7)
        word_idxs.append(idx)
    return ' '.join(idx2word(idx) for idx in word_idxs)

def on_epoch_end(epoch, _):
    example_file.write('\nGenerating text after epoch: %d' % epoch)
    texts = [
        'андрей пришел за мной',
        'война началась',
        'pardon'
    ]
    for text in texts:
        sample = generate_next(text)
        example_file.write('\n %s... -> %s' % (text, sample))
        print('\n %s... -> %s' % (text, sample))


In [11]:
example_file = open('output/war and peace example.txt', 'w')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)
print_text = LambdaCallback(on_epoch_end=on_epoch_end)

history = model.fit(x, y,
                    batch_size=512,
                    epochs=50,
                    verbose=1,
                    callbacks=[print_text, reduce_lr])

Epoch 1/50
180530/180530 [==============================] - 706s 4ms/step - loss: 8.1884


NameError: name 'word2idx' is not defined